In [97]:
# pip install plotly

import plotly.graph_objects as go
import pandas as pd
import numpy as np

df = pd.read_csv('lat_lng_steam_data.csv')
df.head()

,Unnamed: 0,country_code,latitude,longitude,Country,Value
0,0,AR,-38.416097,-63.616672,Argentina,26872
1,1,AT,47.516231,14.550072,Austria,94052
2,2,AU,-25.274398,133.775136,Australia,181386
3,3,AW,12.521110,-69.968338,Aruba,6718
4,4,BA,43.915886,17.679076,Bosnia and Herzegovina,13436


# Choropleth Map

In [96]:
fig = go.Figure(data=go.Choropleth(
    locations = df['Country'],
    locationmode = 'country names',
    z = np.log10(df['Value']),
    marker_line_color='darkgray',
    marker_line_width=0.5,
    text = df.apply(lambda row: f"{row['Country']}<br>{row['Value']} games sold", axis=1),
    hoverinfo = 'text',
    colorscale = 'sunset',
    colorbar=dict(
        tickvals = [1, 4.0, 5.2, 6.1], 
        ticktext = ['1', '10K', '100K', '1.4M']),
    colorbar_tickprefix = '$',
    colorbar_title = 'Number Games Sold',
))

fig.update_layout(
    title_text='Number of Steam Games Sold per Country',
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='equirectangular'
        # for rotatable globe view
        #projection_type='orthographic'
),
    annotations = [dict(
        x=0.55,
        y=0.01,
        xref='paper',
        yref='paper',
        text='Grey countries have no data',
        showarrow = False
    )]
)

fig.update_geos(showocean=True, oceancolor="#e6ffff",
                #if want non rectangle view - gives the curved edges
                projection_type="natural earth")

fig.show()

# Bubble Chart
Still needs work if we want to use this - currently putting add data into one color/limit

In [30]:
df['text'] = df['Country'] + '<br>Games Sold: ' + (df['Value']).astype(str)
limits = [(0,7000),(7001,50000),(50001,100000),(100001,500000),(500001,14000000)]
colors = ["royalblue","crimson","lightseagreen","orange","lightgrey"]
scale = 1000

fig = go.Figure()

for i in range(len(limits)):
    lim = limits[i]
    df_sub = df[lim[0]:lim[1]]
    fig.add_trace(go.Scattergeo(
        locationmode = 'country names',
        lon = df_sub['longitude'],
        lat = df_sub['latitude'],
        text = df_sub['text'],
        marker = dict(
            size = df_sub['Value']/scale,
            color = colors[i],
            line_color='rgb(40,40,40)',
            line_width=0.5,
            sizemode = 'area'
        ),
        name = '{0} - {1}'.format(lim[0],lim[1])))

fig.update_layout(
        title_text = 'Amount of Game Sales per Country <br>(Click legend to toggle traces)',
        showlegend = True,
        geo = dict(
            scope = 'world',
            landcolor = ' #b3ffb3',
        )
    )

fig.show()